In [1]:
import matplotlib.pyplot as plt
import plotly
import numpy as np
import random
import hrr
import math
import time
from plotly.graph_objs import Scatter, Layout, Surface
plotly.offline.init_notebook_mode(connected=True)

In [2]:
def log_transform(error):
    return math.copysign(1.0,error)*math.log(math.fabs(error)+1,2)

In [3]:
def softmax(arr,t=1.0):
    w = np.array(arr)
    e = np.exp(w / t)
    dist = e / np.sum(e)
    return dist

In [31]:
def argmax1(arr_3d,outer,inner):
    max_row = outer[0]
    max_col = inner[0]
    max_x = 0
    #max_value = arr_2d[0,0]
    max_value = arr_3d[outer[0],inner[0],0]
    for row in range(arr_3d.shape[0]):
        if row not in outer:
            continue
        for col in range(arr_3d.shape[1]):
            if col not in inner:
                continue
            for x in range(arr_3d.shape[2]):
                if arr_3d[row,col,x] > max_value:
                    max_value = arr_3d[row,col,x]
                    max_row,max_col,max_x = row,col,x
    return list((max_row,max_col,max_x))

In [4]:
n = 2
num = 5
A = hrr.hrrs(n,num)
B = hrr.hrrs(n,num)

In [5]:
C = hrr.compose(A,B)

In [6]:
A

array([[ 0.48533127,  0.82274783],
       [-0.45205008, -1.67282745],
       [-0.57192024,  0.73195571],
       [-1.36429654, -0.34770737],
       [ 0.83859226,  0.04457619]])

In [7]:
B

array([[-0.2614056 ,  0.16014444],
       [ 1.55497017, -0.08762939],
       [ 0.58734785,  0.47532485],
       [-0.85251089,  0.33468313],
       [ 0.26819364, -0.8377757 ]])

In [8]:
C

array([[ 0.11196283,  0.49144614],
       [ 0.55146005, -0.88022842],
       [ 0.00771381,  0.60364028],
       [-1.10840372, -0.00651212],
       [ 0.55339295, -0.39659975]])

In [9]:
D = A+B

In [10]:
D

array([[ 0.22392567,  0.98289228],
       [ 1.10292009, -1.76045684],
       [ 0.01542761,  1.20728056],
       [-2.21680744, -0.01302424],
       [ 1.10678589, -0.7931995 ]])

In [57]:
def Delayed_Saccade(ntrials,lrate,gamma,td_lambda,temp):
    n = 2
    n_cross_locations = 5
    n_dot_locations = 5
    n_gaze_locations = 5
    n_dot_colors = 4
    n_cross_colors = 4
    #n_saccades = 5 # actions
    
    # working memory chuncks
    n_wm_crosshair_location = 5
    n_wm_dot_location = 5
    n_wm_agent_fixation = 5
    
    # identity vector
    hrr_i = np.zeros(n)
    hrr_i[0] = 1
    
    # saccade hrr
    agent_gaze_location = hrr.hrrs(n,n_gaze_locations)
    agent_gaze_location = np.row_stack((agent_gaze_location,hrr_i))
    
    # crosshair location hrr
    crosshair_location = hrr.hrrs(n,n_cross_locations)
    crosshair_location = np.row_stack((crosshair_location,hrr_i))
    
    # dot location hrr
    dot_location = hrr.hrrs(n,n_dot_locations)
    dot_location = np.row_stack((dot_location,hrr_i))
    
    # color hrr
    dot_color = hrr.hrrs(n,n_dot_colors)
    dot_color = np.row_stack((dot_color,hrr_i))
    
    # crosshair hrr
    crosshair_color = hrr.hrrs(n,n_cross_colors)
    crosshair_color = np.row_stack((crosshair_color,hrr_i))

    # agent fixation working memory hrr
    agent_fixation_wm = hrr.hrrs(n,n_wm_agent_fixation)
    agent_fixation_wm = np.row_stack((agent_fixation_wm,hrr_i))
    
    # dot location working memory hrr
    dot_location_wm = hrr.hrrs(n,n_wm_dot_location)
    dot_location_wm = np.row_stack((dot_location_wm,hrr_i))
    
    # crosshair location working memory hrr
    crosshair_location_wm = hrr.hrrs(n,n_wm_crosshair_location)
    crosshair_location_wm = np.row_stack((crosshair_location_wm,hrr_i))
    
    #tick1 = time.time()
    #print('starting precompute ...')
    # precompute hrrs
    
    #tick2 = time.time()
    #print("time spent precomputing:",(tick2-tick1),"seconds")
    #print(representation.shape)
    # weight vector and bias
    W = hrr.hrr(n)
    bias = 1
    
    # epsilon for e-soft policy
    epsilon = .1
    
    # temperature for softmax
    t = temp
    
    # eligibility trace
    eligibility = np.zeros(n)
    center = 4
    correct_response_count = 0
    count = 0
    for trial in range(1):#ntrials):
        count += 1
        r = 0
        flag_count = 0 # used to determine if agent completed all stages correct
        
        eligibility = np.zeros(n)
        ######## just the crosshair on the screen ###########
        representation = hrr.convolve(agent_gaze_location,hrr.convolve(crosshair_location[center],dot_location[-1]))
        wm = hrr.oconvolve(agent_fixation_wm,hrr.oconvolve(dot_location_wm,crosshair_location_wm))
        representation = hrr.oconvolve(representation,wm)
        representation = representation.reshape((n_gaze_locations+1,n_wm_dot_location+1,n_wm_crosshair_location+1,
                                                n_wm_agent_fixation+1,n))
        #print(representation.shape)
        values = np.dot(representation,W) + bias
        #print(values)
        #print(values.shape)
        sm_prob = softmax(values,t)
        #print(sm_prob)
    
        wm_saccade = np.unravel_index(np.argmax(sm_prob),sm_prob.shape)
        fixation = wm_saccade[0]
        wm_agent = wm_saccade[1]
        wm_crosshair = wm_saccade[2]
        wm_dot = wm_saccade[3]
        #print(wm_cross,wm_agent,action)
        #epsilon = .2
        if random.random() < epsilon:
            fixation = random.randrange(n_gaze_locations)
        
        value = values[fixation,wm_agent,wm_crosshair,wm_dot]
        pvalue = value
        pfixation = fixation
        
        #eligibility = representation[fixation,wm_agent,wm_crosshair,wm_dot,:] + td_lambda*eligibility
        trace1 = hrr.convolve(agent_gaze_location[fixation],agent_fixation_wm[wm_agent])
        trace2 = hrr.convolve(crosshair_location_wm[wm_crosshair],dot_location_wm[wm_dot])
        e = hrr.convolve(trace1,trace2)
        eligibility = eligibility + td_lambda*eligibility
        if fixation == center:
            flag_count += 1
            r = 0
        
        #print(fixation,end=" ")
        ####################
        
        ########## crosshair and colored dot on the screen #######
        dot_loc = random.randint(0,3)
        representation = hrr.convolve(agent_gaze_location,hrr.convolve(crosshair_location[center],dot_location[dot_loc]))
        wm = hrr.oconvolve(agent_fixation_wm,hrr.oconvolve(dot_location_wm,crosshair_location_wm))
        representation = hrr.oconvolve(representation,wm)
        representation = representation.reshape((n_gaze_locations+1,n_wm_dot_location+1,n_wm_crosshair_location+1,
                                                n_wm_agent_fixation+1,n))
        #print(representation.shape)
        values = np.dot(representation,W) + bias
        sm_prob = softmax(values,t)
        wm_saccade = np.unravel_index(np.argmax(sm_prob),sm_prob.shape)
        fixation = wm_saccade[0]
        wm_agent = wm_saccade[1]
        wm_crosshair = wm_saccade[2]
        wm_dot = wm_saccade[3]
        
        #epsilon = .2
        if random.random() < epsilon:
            fixation = random.randrange(n_gaze_locations)
        
        value = values[fixation,wm_agent,wm_crosshair,wm_dot]
        # update error and weights
        error = (r+gamma*value) - pvalue
        W += lrate*log_transform(error)*eligibility
        
        pvalue = value
        pfixation = fixation
        
        #eligibility = representation[fixation,wm_agent,wm_crosshair,wm_dot,:] + td_lambda*eligibility
        trace1 = hrr.convolve(agent_gaze_location[fixation],agent_fixation_wm[wm_agent])
        trace2 = hrr.convolve(crosshair_location_wm[wm_crosshair],dot_location_wm[wm_dot])
        e = hrr.convolve(trace1,trace2)
        eligibility = e + td_lambda*eligibility
        if fixation == center:
            flag_count += 1
            r = 0
        
        #print(fixation,end=" ")
        ######################
        
        ####### just the crosshair on the screen ################
        
        representation = hrr.convolve(agent_gaze_location,hrr.convolve(crosshair_location[center],dot_location[-1]))
        wm = hrr.oconvolve(agent_fixation_wm,hrr.oconvolve(dot_location_wm,crosshair_location_wm))
        representation = hrr.oconvolve(representation,wm)
        representation = representation.reshape((n_gaze_locations+1,n_wm_dot_location+1,n_wm_crosshair_location+1,
                                                n_wm_agent_fixation+1,n))
        #print(representation.shape)
        values = np.dot(representation,W) + bias
        #print(values)
        #print(values.shape)
        sm_prob = softmax(values,t)
        #print(sm_prob)
        wm_saccade = np.unravel_index(np.argmax(sm_prob),sm_prob.shape)
        fixation = wm_saccade[0]
        wm_agent = wm_saccade[1]
        wm_crosshair = wm_saccade[2]
        wm_dot = wm_saccade[3]
        #print(wm_cross,wm_agent,action)
        
        #epsilon = .2
        if random.random() < epsilon:
            fixation = random.randrange(n_gaze_locations)
        
        value = values[fixation,wm_agent,wm_crosshair,wm_dot]
        error = (r+gamma*value) - pvalue
        W += lrate*log_transform(error)*eligibility
        
        pvalue = value
        pfixation = fixation
        #eligibility = representation[fixation,wm_agent,wm_crosshair,wm_dot,:] + td_lambda*eligibility
        trace1 = hrr.convolve(agent_gaze_location[fixation],agent_fixation_wm[wm_agent])
        trace2 = hrr.convolve(crosshair_location_wm[wm_crosshair],dot_location_wm[wm_dot])
        e = hrr.convolve(trace1,trace2)
        eligibility = e + td_lambda*eligibility
        
        if fixation == center:
            flag_count += 1
            r = 0
        
        #print(fixation,end=" ")
        ##############################
        
        ######## remove crosshair on the screen ################
        representation = hrr.convolve(agent_gaze_location,hrr.convolve(crosshair_location[-1],dot_location[-1]))
        
        values = np.dot(representation,W) + bias
        sm_prob = softmax(values,t)
        #print(sm_prob.shape)
        fixation = np.argmax(sm_prob)
        
        # updated info
        '''
        if random.random() < epsilon:
            fixation = random.randrange(n_gaze_locations)
        
        if fixation == dot_loc:
            flag_count += 1
            r = 1
           
        if flag_count == 4:
            r = 4
        '''
        value = values[fixation]
        error = (r+gamma*value) - pvalue
        #error = r - value
        W += lrate*log_transform(error)*eligibility
        
        pvalue = value
        pfixation = fixation
        #eligibility = representation[fixation,:] + td_lambda*eligibility
        e = hrr.convolve(agent_gaze_location[fixation],hrr.convolve(crosshair_location[-1],dot_location[-1]))
        eligibility = e + td_lambda*eligibility
        
        #############################
        
        #print('dot_loc:','(',dot_loc,')',end = ' ')
        #epsilon = .4
        if random.random() < epsilon:
            fixation = random.randrange(n_gaze_locations)
        
        if fixation == dot_loc:
            flag_count += 1 
            r = 0
            
        if flag_count == 4:
            r = 1
            print('correct')
            correct_response_count += 1 # used to track performance
        value = values[fixation]
        error = r - value
        W += lrate*log_transform(error)*eligibility
        
        #print(fixation)
        if trial % 100 == 0:
            performance = correct_response_count/count
            print('Trial:',trial)
            print('Performance:',performance)
            count = 0
            correct_response_count = 0
    try:   
        performance = correct_response_count/count
        print('Trial:',trial)
        print('Performance:',performance)
        count = 0
        correct_response_count = 0
    except ZeroDivisionError:
        print('Trial:',trial)
        print('Performance:',0)
    

In [58]:
Delayed_Saccade(100000,.1,.9,.8,.1)
# (num trials, learning rate, discount factor(gamma), lambda, temperature)

Trial: 0
Performance: 0.0
Trial: 0
Performance: 0
